In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
import os
import random
import pandas as pd
import numpy as np
import cv2
from keras.utils import np_utils

In [3]:
IMAGE_SIZE = 100
CLASS = ["angry","disgust","fear","happy","neutral","sad","surprise"]
train_dir = "images/train"
test_dir = "images/test"

In [4]:
angry_path = os.path.join(train_dir,CLASS[0])
os.listdir(angry_path)

['24283.jpg',
 '29810.jpg',
 '10242.jpg',
 '9535.jpg',
 '19369.jpg',
 '7046.jpg',
 '34333.jpg',
 '11102.jpg',
 '5967.jpg',
 '10894.jpg',
 '8930.jpg',
 '12049.jpg',
 '17696.jpg',
 '28811.jpg',
 '34944.jpg',
 '7187.jpg',
 '33542.jpg',
 '8423.jpg',
 '4853.jpg',
 '18065.jpg',
 '27516.jpg',
 '33682.jpg',
 '15700.jpg',
 '360.jpg',
 '7779.jpg',
 '1513.jpg',
 '27668.jpg',
 '28647.jpg',
 '25634.jpg',
 '16253.jpg',
 '35069.jpg',
 '12270.jpg',
 '28162.jpg',
 '31077.jpg',
 '18912.jpg',
 '29868.jpg',
 '5603.jpg',
 '28089.jpg',
 '4501.jpg',
 '30526.jpg',
 '16638.jpg',
 '5246.jpg',
 '5129.jpg',
 '33064.jpg',
 '16919.jpg',
 '3702.jpg',
 '19344.jpg',
 '6898.jpg',
 '19242.jpg',
 '26263.jpg',
 '30852.jpg',
 '14137.jpg',
 '32431.jpg',
 '29503.jpg',
 '18806.jpg',
 '31586.jpg',
 '29544.jpg',
 '19766.jpg',
 '4438.jpg',
 '6952.jpg',
 '1397.jpg',
 '28885.jpg',
 '13517.jpg',
 '11377.jpg',
 '32423.jpg',
 '26449.jpg',
 '26773.jpg',
 '25611.jpg',
 '7096.jpg',
 '30609.jpg',
 '1098.jpg',
 '29348.jpg',
 '15112.jpg',


In [5]:
def create_data(dir_name) :
    data_set = []
    
    for cl in CLASS :
        cl_path = os.path.join(dir_name,cl)
        class_num = CLASS.index(cl)
        for img in os.listdir(cl_path) :
            try :
                img_array = cv2.imread(os.path.join(cl_path,img))
                img_array = cv2.resize(img_array,(IMAGE_SIZE,IMAGE_SIZE))
                gray_img  = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                data_set.append([gray_img, class_num])
            except :
                pass
            
        random.shuffle(data_set)
        
    return data_set
            
        

In [6]:
train_data = create_data(train_dir)
test_data = create_data(test_dir)

In [7]:
#shuffle the train data
random.shuffle(train_data)
print(len(train_data))

#shuffle the test data
random.shuffle(test_data)
print(len(test_data))

28821
7066


In [8]:
train_data

[[array([[200, 202, 206, ..., 143, 180, 198],
         [205, 207, 210, ..., 134, 174, 193],
         [216, 217, 220, ..., 116, 160, 181],
         ...,
         [ 49,  48,  45, ...,  13,  11,  11],
         [ 50,  47,  42, ...,  12,  11,  11],
         [ 50,  47,  41, ...,  11,  12,  12]], dtype=uint8),
  3],
 [array([[ 83,  80,  75, ...,  90,  92,  93],
         [ 86,  83,  77, ...,  91,  93,  93],
         [ 94,  90,  81, ...,  93,  94,  94],
         ...,
         [ 79,  84,  97, ..., 241, 237, 235],
         [ 84,  90, 102, ..., 241, 241, 241],
         [ 87,  92, 104, ..., 241, 243, 244]], dtype=uint8),
  0],
 [array([[156, 155, 154, ..., 177, 177, 177],
         [156, 155, 155, ..., 177, 177, 177],
         [155, 155, 156, ..., 177, 177, 177],
         ...,
         [138, 137, 133, ..., 166, 167, 167],
         [125, 124, 123, ..., 167, 166, 166],
         [119, 119, 118, ..., 167, 166, 166]], dtype=uint8),
  3],
 [array([[243, 219, 167, ..., 120, 171, 194],
         [241, 218, 1

In [9]:
X_train = []
y_train = []

for features, label in train_data:
    X_train.append(features)
    y_train.append(label)

In [10]:
X_test = []
y_test = []

for features, label in test_data:
    X_test.append(features)
    y_test.append(label)

In [11]:
(np.array(X_train).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)).shape

(28821, 100, 100, 1)

In [12]:
X_train = np.array(X_train).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
y_train = np.array(y_train)

X_train = X_train/255.0

In [13]:
X_test = np.array(X_test).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
y_test = np.array(y_test)

X_test = X_test/255.0

In [14]:
num_classes = len(os.listdir(train_dir))
num_classes

7

In [15]:
X_train.shape[1:]

(100, 100, 1)

In [16]:
# convert class labels to one-hot encoding
y_train = np_utils.to_categorical(y_train, num_classes)

y_test = np_utils.to_categorical(y_test, num_classes)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [18]:
# building the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 98, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 49, 49, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 49, 49, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       7

In [20]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
901/901 [==============================] - 571s 547ms/step - loss: 1.7794 - accuracy: 0.3251 - val_loss: 1.4252 - val_accuracy: 0.4451
Epoch 2/10
901/901 [==============================] - 387s 429ms/step - loss: 1.2535 - accuracy: 0.5199 - val_loss: 1.4198 - val_accuracy: 0.4652
Epoch 3/10
901/901 [==============================] - 384s 426ms/step - loss: 1.0866 - accuracy: 0.5863 - val_loss: 1.1962 - val_accuracy: 0.5502
Epoch 4/10
901/901 [==============================] - 385s 427ms/step - loss: 0.9433 - accuracy: 0.6497 - val_loss: 1.1573 - val_accuracy: 0.5742
Epoch 5/10
901/901 [==============================] - 386s 429ms/step - loss: 0.7575 - accuracy: 0.7197 - val_loss: 1.2337 - val_accuracy: 0.5804
Epoch 6/10
901/901 [==============================] - 386s 429ms/step - loss: 0.5843 - accuracy: 0.7897 - val_loss: 1.2838 - val_accuracy: 0.5914
Epoch 7/10
901/901 [==============================] - 390s 433ms/step - loss: 0.4224 - accuracy: 0.8500 - val_loss: 1.4329 -

In [21]:
model.save("models/model1.h5")